### =======================================================================
### IMPORTACIÓN DE DATOS.
### =======================================================================

In [1]:
## IMPORTACIÓN GENERAL DE LIBRERIAS Y VISUALIZACIÓN DE DATOS (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as DT
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')
plt.style.use('default') 
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (15, 10)
# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier

In [2]:
## OBTENEMOS TODA LA INFORMACIÓN DEL SET DE ENTRENAMIENTO.
training = pd.read_csv('labels_training_set.csv', encoding = 'utf-8')
## OBTENEMOS TODA LA INFORMACIÓN DE LOS DIFERENTES EVENTOS.
eventos = pd.read_csv('events_up_to_01062018.csv', encoding = 'utf-8')
## OBTENEMOS TODA LA INFORMACIÓN A TESTEAR.
test = pd.read_csv('trocafone_kaggle_test.csv', encoding = 'utf-8')

In [3]:
## PASAMOS LAS COLUMNAS QUE TIENEN UNA CANTIDAD DE VALORES LIMITADA A UN TIPO CATEGORY
eventos['event'] = eventos['event'].astype('category')
eventos['condition'] = eventos['condition'].astype('category')
eventos['storage'] = eventos['storage'].astype('category')
eventos['search_engine'] = eventos['search_engine'].astype('category')
eventos['channel'] = eventos['channel'].astype('category')
eventos['new_vs_returning'] = eventos['new_vs_returning'].astype('category')
eventos['device_type'] = eventos['device_type'].astype('category')
eventos['color'] = eventos['color'].astype('category')
eventos['region'] = eventos['region'].astype('category')
eventos['country'] = eventos['country'].astype('category')
eventos['operating_system_version'] = eventos['operating_system_version'].astype('category')
eventos['city'] = eventos['city'].astype('category')
eventos['browser_version'] = eventos['browser_version'].astype('category')
eventos['screen_resolution'] = eventos['screen_resolution'].astype('category')
eventos['model'] = eventos['model'].astype('category')
eventos['timestamp'] = pd.to_datetime(eventos['timestamp'], infer_datetime_format=True)

### =======================================================================
### ARMADO DE FEATURES.
### =======================================================================

In [4]:
## ORDENAMOS LOS DATOS ṔOR PERSONAS EN PRIMER LUGAR Y TIEMPO EN SEGUNDO.
eventos.sort_values(['person', 'timestamp'], ascending=[True, True], inplace=True)
## DIFERENCIAMOS EN TRES COLUMNAS DIFERENTES EL DIA, MES Y AÑO.
eventos['mes'] = eventos['timestamp'].dt.month
eventos['dia'] = eventos['timestamp'].dt.day
eventos['hora'] = eventos['timestamp'].dt.hour
## ARMAMOS UNA COLUMNA PARA EL DÍA DE LA SEMANA COMO NOMBRE.
eventos['diasemana'] = eventos['timestamp'].dt.weekday_name
## PONEMOS LOS NOMBRES DE MANERA MÁS PROLIJA PARA LOS GRÁFICOS.
eventos.loc[eventos.diasemana.str.contains('Monday', na=False), 'diasemana'] = 'lunes'
eventos.loc[eventos.diasemana.str.contains('Tuesday', na=False), 'diasemana'] = 'martes'
eventos.loc[eventos.diasemana.str.contains('Wednesday', na=False), 'diasemana'] = 'miercoles'
eventos.loc[eventos.diasemana.str.contains('Thursday', na=False), 'diasemana'] = 'jueves'
eventos.loc[eventos.diasemana.str.contains('Friday', na=False), 'diasemana'] = 'viernes'
eventos.loc[eventos.diasemana.str.contains('Saturday', na=False), 'diasemana'] = 'sabado'
eventos.loc[eventos.diasemana.str.contains('Sunday', na=False), 'diasemana'] = 'domingo'
# DEFINIMOS UNA LÓGICA PARA INDICAR SI EL DÍA EN QUE SE EJECUTA EL EVENTO ES FIN DE SEMANA.
eventos['finde'] = 0
eventos.loc[(eventos.diasemana.str.contains('DOM', na=False) | eventos.diasemana.str.contains('SAB', na=False)), 'finde'] = 1
# DEFINIMOS EL MES COMO NOMBRE PARA FACILITAR LAS COLUMNAS
eventos['mesMayus'] = ''
eventos.loc[eventos.mes == 1, 'mesMayus'] = 'enero'
eventos.loc[eventos.mes == 2, 'mesMayus'] = 'febrero'
eventos.loc[eventos.mes == 3, 'mesMayus'] = 'marzo'
eventos.loc[eventos.mes == 4, 'mesMayus'] = 'abril'
eventos.loc[eventos.mes == 5, 'mesMayus'] = 'mayo'
eventos.loc[eventos.mes == 6, 'mesMayus'] = 'junio'
eventos.loc[eventos.mes == 7, 'mesMayus'] = 'julio'
eventos.loc[eventos.mes == 8, 'mesMayus'] = 'agosto'
eventos.loc[eventos.mes == 9, 'mesMayus'] = 'septiembre'
eventos.loc[eventos.mes == 10, 'mesMayus'] = 'octubre'
eventos.loc[eventos.mes == 11, 'mesMayus'] = 'noviembre'
eventos.loc[eventos.mes == 12, 'mesMayus'] = 'diciembre'
# ARMAMOS UNA LÓGICA PARA SEGMENTAR LAS FRANJAS HORARIAS.
# MADRUGADA de 00 a 06
eventos['hora_madrugada'] = 0
eventos.loc[((eventos.hora > -1) & (eventos.hora < 7)), 'hora_madrugada'] = 1
# MAÑANA de 07 a 11
eventos['hora_mañana'] = 0
eventos.loc[((eventos.hora > 6) & (eventos.hora < 12)), 'hora_mañana'] = 1
# ALMUERZO de 12 a 13
eventos['hora_almuerzo'] = 0
eventos.loc[((eventos.hora > 11) & (eventos.hora < 14)), 'hora_almuerzo'] = 1
# TARDE de 14 a 18
eventos['hora_tarde'] = 0
eventos.loc[((eventos.hora > 13) & (eventos.hora < 19)), 'hora_tarde'] = 1
# NOCHE de 19 a 23
eventos['hora_noche'] = 0
eventos.loc[((eventos.hora > 18) & (eventos.hora < 24)), 'hora_noche'] = 1
# TRANSFORMAMOS EN CATEGÓRICAS EL DÍA DE LA SEMANA Y EL MES.
eventos['diasemana'] = eventos['diasemana'].astype('category')
eventos['mesMayus'] = eventos['mesMayus'].astype('category')

In [5]:
eventos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2341681 entries, 1507286 to 1504503
Data columns (total 34 columns):
timestamp                   datetime64[ns]
event                       category
person                      object
url                         object
sku                         float64
model                       category
condition                   category
storage                     category
color                       category
skus                        object
search_term                 object
staticpage                  object
campaign_source             object
search_engine               category
channel                     category
new_vs_returning            category
city                        category
region                      category
country                     category
device_type                 category
screen_resolution           category
operating_system_version    category
browser_version             category
mes                         int64
dia

In [6]:
# COLUMNAS DONDE POR CADA REGISTRO SABEMOS QUE TENEMOS UN VALOR (SIEMPRE PRESENTES)
dummies_diasemana = pd.get_dummies(eventos['diasemana'], drop_first=False)
eventos = pd.concat([eventos, dummies_diasemana], axis=1)
dummies_mesMayus = pd.get_dummies(eventos['mesMayus'], drop_first=False)
eventos = pd.concat([eventos, dummies_mesMayus], axis=1)
dummies_event = pd.get_dummies(eventos['event'], drop_first=False)
eventos = pd.concat([eventos, dummies_event], axis=1)

In [7]:
## ESTAS COLUMNAS NO APARECEN SIEMPRE, PUESTO QUE SOLO APARECEN SEGÚN EL TIPO DE EVENTO.
dummies_color = pd.get_dummies(eventos['color'], drop_first=False)
eventos = pd.concat([eventos, dummies_color], axis=1)
dummies_model = pd.get_dummies(eventos['model'], drop_first=False)
eventos = pd.concat([eventos, dummies_model], axis=1)
dummies_condition = pd.get_dummies(eventos['condition'], drop_first=False)
eventos = pd.concat([eventos, dummies_condition], axis=1)
dummies_storage = pd.get_dummies(eventos['storage'], drop_first=False)
eventos = pd.concat([eventos, dummies_storage], axis=1)

In [ ]:
columnas_filtrar = list(eventos.select_dtypes(include=['int','float64','uint8']).columns)

columnas_filtrar.remove('sku')
columnas_filtrar.remove('staticpage')
columnas_filtrar.append('person')
asd = eventos.loc[:, eventos.columns.isin(columnas_filtrar)]
columnas_filtrar.remove('person')

eventos_agrupados = asd.groupby('person')[columnas_filtrar]


#asd.head()
#grouped_df = eventos.groupby('person').mean().reset_index()
#entrenar = pd.](grouped_df)

### =======================================================================
### ENTRENAMIENTO Y PREDICCIÓN.
### =======================================================================

In [9]:
train_completo = pd.merge(training, eventos, on='person', how='left')
train_completo.dropna(subset=['person']) 
train_completo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1171886 entries, 0 to 1171885
Columns: 342 entries, person to 8GB
dtypes: category(17), datetime64[ns](1), float64(1), int64(10), object(6), uint8(307)
memory usage: 537.7+ MB


In [10]:
y = pd.factorize(train_completo['label'])[0]
train_completo = train_completo[['mes','dia','hora','finde','hora_madrugada','hora_mañana','hora_almuerzo','hora_tarde','hora_noche','domingo','jueves','lunes','martes','miercoles','sabado','viernes','abril','enero','febrero','marzo','mayo']]
# VER COMO OBTENER LAS COLUMNAS 
# ESPECÍFICAS QUE QUEREMOS TRABAJAR.
features = train_completo.columns

In [11]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [12]:
# Creamos un clasificador con Random Forest..
clf = RandomForestClassifier(n_jobs=2, random_state=0)
# Entrenamos.
clf.fit(train_completo[features], y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [13]:
# Predecimos.
eventos['label'] = 0
clf.predict(eventos[features])

array([0, 0, 0, ..., 0, 0, 0])

In [14]:
# REVISAR. todos dan iguales.
prueba = clf.predict_proba(eventos[features])[0:10000]

In [15]:
y = np.unique(prueba)

In [16]:
y

array([0.        , 0.00263289, 0.00326545, ..., 0.99673455, 0.99736711,
       1.        ])